In [2]:
import pandas as pd
import numpy as np
import requests
import beautifulsoup

ModuleNotFoundError: No module named 'beautifulsoup'

In [3]:
url = 'https://www.erowid.org/experiences/exp.php?ID=89042'
page = requests.get(url)

In [4]:
page.content

b'<html>\n<head>\n<title>MDMA, LSD & Cannabis - Erowid Exp - \'Some Growing Up to Do\'</title>\n<meta name="description" content="An Experience with MDMA, LSD & Cannabis. \'Some Growing Up to Do\' by thingummajig">\n<meta name="keywords" content="Experience Report Vaults drug trip reports stories descriptions">\n<link href="/includes/general_default.css" rel="stylesheet" type="text/css">\n<link href="includes/exp_view.css" rel="stylesheet" type="text/css" id="main_css" >\n<link href="includes/exp_view_light_on_dark.css" rel="stylesheet" type="text/css" id="color_css" >\n<script type="text/javascript" src="/includes/javascript/e_cookie_lib.js"></script>\n<script type="text/javascript" src="/experiences/includes/exp_view.js"></script>\n</head>\n<body onload="expSetColors();" >\n\n<table border="0" cellpadding="0" cellspacing="0" align="center"><tr><td>\n<img src="/experiences/images/exp_header_blk.gif" width="707" height=75 border="0" usemap="#exp_header"/>\n</td></tr><tr height="3"><td>